In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(0, '../src')
from classifier_results_metrics import output_metrics_for_augmentation, output_metrics_folds

In [ ]:
baseline_path = "../outputs_rotation_0/fold"
df_baseline = output_metrics_for_augmentation(baseline_path)
df_baseline["dim"] = "baseline"
df_baseline

In [ ]:
hue_order = ["unet", "clss", "base"]
# bands = ["C", "S", "X"]
bands = df_baseline["band"].unique()
bands = sorted(list(bands))
n_bands = len(bands)
f, a = plt.subplots(n_bands, 2, figsize=(15, n_bands*5))
a = a.reshape(n_bands, 2)
for i, band in enumerate(bands):
    sns.boxplot(data=df_baseline.query(f"band == '{band}'"), ax=a[i, 0], x="test_dir",
                y="mcc", hue="model", hue_order=hue_order)
    a[i, 0].set_title(band)
    a[i, 0].axhline(y=0, c="k", ls='--',linewidth=0.5)
    
    sns.barplot(data=df_baseline.query(f"band == '{band}'"), ax=a[i, 1], x="test_dir",
                y="mcc", hue="model", hue_order=hue_order)
    a[i, 1].set_title(band)
    a[i, 1].axhline(y=0, c="k", ls='--',linewidth=0.5)
plt.show()

In [ ]:
# f, a = plt.subplots(1, 1, figsize=(13, 8))
# sns.boxplot(data=df_all.query("model == 'unet'"), ax=a, x="test_dir", y="mcc", hue="label")
# plt.show()

f, a = plt.subplots(n_bands, 2, figsize=(15, n_bands*5))
a = a.reshape(n_bands, 2)
for i, band in enumerate(bands):
    sns.boxplot(data=df_baseline.query(f"model == 'unet' and band == '{band}'"),
                ax=a[i, 0], x="test_dir", y="mcc", hue="label")
    a[i, 0].set_title(band)
    a[i, 0].axhline(y=0, c="k", ls='--',linewidth=0.5)
    
    sns.barplot(data=df_baseline.query(f"model == 'unet' and band == '{band}'"),
                ax=a[i, 1], x="test_dir", y="mcc", hue="label")
    a[i, 1].set_title(band)
    a[i, 1].axhline(y=0, c="k", ls='--',linewidth=0.5)
plt.show()

In [ ]:
rotations = [f"rotation_{x}" for x in [0, 90]]
n = len(rotations)
f, a = plt.subplots(n, 1, figsize=(13, n*4))
for i, rotation in enumerate(rotations):
    sns.barplot(data=df_baseline.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="band", y="mcc", hue="label")
    a[i].set_title(f"Rotation = {rotation.split('_')[1]}")
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
rotations = [f"rotation_{x}" for x in [0, 90]]
n = len(rotations)
f, a = plt.subplots(n, 1, figsize=(13, n*5))
for i, rotation in enumerate(rotations):
    sns.boxplot(data=df_baseline.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="band", y="mcc", hue="label")
    sns.stripplot(data=df_baseline.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="band", y="mcc", hue="label", color='k', dodge=True, jitter=0.3)
    a[i].set_title(rotation)
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
    handles, labels = a[i].get_legend_handles_labels()
    l = a[i].legend(handles[0:7], labels[0:7])
plt.show()

In [ ]:
import scipy.stats as st
def ci_95(data):
    ci = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    interval = (ci[1] - ci[0]) / 2
    if interval is np.nan:
        return 0
    else:
        return interval

df_baseline.query("model == 'unet'").pivot_table(index="label",
                                                 columns=["band", "test_dir"],
                                                 values="mcc",
                                                 aggfunc=["mean", ci_95])

In [ ]:
df_baseline["band"].value_counts()

In [ ]:
df_baseline["test_dir"].value_counts()

In [ ]:
df_baseline["model"].value_counts()

In [ ]:
df_baseline["fold"].value_counts()